In [50]:
from langchain_community.document_loaders import PyPDFLoader 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import format_document
from langchain.schema import StrOutputParser
import os

In [51]:
os.environ["GOOGLE_API_KEY"] = open('passgooglestudio.txt', 'r').read()

https://www.bcb.gov.br/publicacoes/atascopom

In [52]:
# Coleta de dados ----
import requests
ultima_ata = requests.get(
  "https://www.bcb.gov.br/api/servico/sitebcb/copom/atas?quantidade=1"
  ).json()
no_reuniao = str(ultima_ata["conteudo"][0]["nroReuniao"])
data_refer = str(ultima_ata["conteudo"][0]["dataReferencia"]).replace("-", "")

url_ata = f"https://www.bcb.gov.br/content/copom/atascopom/Copom{no_reuniao}-not{data_refer+no_reuniao}.pdf"

url_ata

'https://www.bcb.gov.br/content/copom/atascopom/Copom261-not20240320261.pdf'

In [53]:
# Coleta de dados ----
pdf = PyPDFLoader(url_ata)### "https://www.bcb.gov.br/content/copom/atascopom/Copom261-not20240320261.pdf")
ata = pdf.load_and_split()


# Definição do modelo LLM e Engenharia de prompt ----
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

doc_prompt = PromptTemplate.from_template("{page_content}") # extrai texto do PDF

llm_prompt = PromptTemplate.from_template("""
Você é um economista chefe experiente em análise de política monetária, tendo
trabalhado em fundos de investimentos e grandes bancos, além de ser PhD em
economia.

Sua tarefa é ler a ata a seguir da Reunião do Comitê de Política Monetária (Copom)
do Banco Central do Brasil e classificar o documento em hawkish ou dovish:

"{ata}"

Classificação:""") # prompt para o Gemini

# Combinar prompt, texto da ata e modelo
classificador = (
    {"ata": lambda ata: "\n\n".join(
      format_document(pagina, doc_prompt) for pagina in ata
      )}
    | llm_prompt
    | llm
    | StrOutputParser()
)

classificador.invoke(ata[2:8])

'Hawkish'

In [54]:
#  Bibliotecas ----
# from shiny import App, reactive, render, ui
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
# import os

In [56]:
pdf = PyPDFLoader('data/carta-fev-24.pdf')
                  #https://www.dynamo.com.br/uploads/a615cb4e89402e053db8cf63256714814c312510.pdf')
texto = pdf.load_and_split()

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
doc_prompt = PromptTemplate.from_template("{page_content}") # extrai texto do PDF
prompt = PromptTemplate.from_template(
            """Escreva um sumário do texto a seguir delimitado com 3 crases.
            Retorne sua resposta em 3 ou 5 marcadores com uma breve descrição dos
            pontos chave do texto.

            ```{TEXTO}```

            SUMÁRIO:""")


llm_chain = LLMChain(llm = llm, prompt = prompt)
modelo = StuffDocumentsChain(
            llm_chain = llm_chain,
            document_variable_name = "TEXTO"
            )

resultado = modelo.run(texto)

print(resultado)

- **Cenário Macroeconômico:**
    - Dinâmicas de reprecificação de juros nos EUA e inflação persistente na Europa.
    - Crescimento econômico projetado para 1% no Brasil em 2024, com cortes de juros previstos para meados do ano.

- **Projeções Econômicas:**
    - PIB do Brasil projetado para crescer 1% em 2024, com inflação de 3,6%.
    - Taxa Selic estimada em 9,25% em meados de 2024.

- **Mercado de Crédito:**
    - Otimismo para 2024, com afrouxamento da política monetária global e redução da Selic.
    - Potencial de ganhos de capital reduzido devido à compressão anterior dos spreads.


In [ ]:
os.environ["OPENAI_API_KEY"] = open('passopenai.txt', 'r').read()

pdf = PyPDFLoader('https://aluno.analisemacro.com.br/download/59224/?tmstv=1710776237')
                  #https://www.dynamo.com.br/uploads/a615cb4e89402e053db8cf63256714814c312510.pdf')
texto = pdf.load_and_split()

llm = ChatOpenAI(temperature = 0, model_name = "gpt-3.5-turbo")

prompt = PromptTemplate.from_template(
            """Escreva um sumário do texto a seguir delimitado com 3 crases.
            Retorne sua resposta em 3 ou 5 marcadores com uma breve descrição dos
            pontos chave do texto.

            ```{TEXTO}```

            SUMÁRIO:""")

llm_chain = LLMChain(llm = llm, prompt = prompt)
modelo = StuffDocumentsChain(
            llm_chain = llm_chain,
            document_variable_name = "TEXTO"
            )

resultado = modelo.run(texto)
